# Revenue Risk, Concentration & Growth Levers in Amazon India Sales

### Decision Intelligence Project

**Objective:**  
To identify revenue drivers, concentration risk, and high-impact growth levers using Amazon India sales data, and to recommend data-driven business actions.

**Key Questions:**
- Where is revenue coming from and how concentrated is it?
- How much revenue is lost due to cancellations and returns?
- Which lever delivers the highest revenue impact: volume, value, or leakage reduction?


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unlock-profits-with-e-commerce-sales-data/Amazon Sale Report.csv
/kaggle/input/unlock-profits-with-e-commerce-sales-data/Cloud Warehouse Compersion Chart.csv
/kaggle/input/unlock-profits-with-e-commerce-sales-data/May-2022.csv
/kaggle/input/unlock-profits-with-e-commerce-sales-data/P  L March 2021.csv
/kaggle/input/unlock-profits-with-e-commerce-sales-data/International sale Report.csv
/kaggle/input/unlock-profits-with-e-commerce-sales-data/Expense IIGF.csv
/kaggle/input/unlock-profits-with-e-commerce-sales-data/Sale Report.csv


## 1. Business Context & Problem Statement

Amazon India has observed concerns around revenue sustainability due to order cancellations, returns, and reliance on a limited number of high-performing categories.  
The goal of this analysis is to move beyond descriptive reporting and quantify **revenue risk, concentration, and controllable growth levers**.


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/unlock-profits-with-e-commerce-sales-data")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/unlock-profits-with-e-commerce-sales-data


## 2. Data Loading & Overview

In [3]:
df = pd.read_csv("/kaggle/input/unlock-profits-with-e-commerce-sales-data/Amazon Sale Report.csv")
df.head(5)

/tmp/ipykernel_17/3605248582.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/unlock-profits-with-e-commerce-sales-data/Amazon Sale Report.csv")


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


## 3. Data Cleaning & Assumptions

### Cleaning Decisions:
- Standardized missing values
- Converted date fields to datetime
- Selected only business-relevant columns

### Business Assumptions:
- Cancelled and returned orders do **not** contribute to realized revenue
- Revenue analysis is based only on successfully fulfilled orders

In [4]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

In [5]:
pd.set_option('display.max_columns',None)
df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [6]:
required_cols = ['Date','Status','Fulfilment','ship-service-level','Category','Qty','Amount','ship-city','ship-state']
wdf = df[required_cols].copy()
wdf.head()

,Date,Status,Fulfilment,ship-service-level,Category,Qty,Amount,ship-city,ship-state
0,04-30-22,Cancelled,Merchant,Standard,Set,0,647.62,MUMBAI,MAHARASHTRA
1,04-30-22,Shipped - Delivered to Buyer,Merchant,Standard,kurta,1,406.00,BENGALURU,KARNATAKA
2,04-30-22,Shipped,Amazon,Expedited,kurta,1,329.00,NAVI MUMBAI,MAHARASHTRA
3,04-30-22,Cancelled,Merchant,Standard,Western Dress,0,753.33,PUDUCHERRY,PUDUCHERRY
4,04-30-22,Shipped,Amazon,Expedited,Top,1,574.00,CHENNAI,TAMIL NADU


In [7]:
wdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Date                128975 non-null  object 
 1   Status              128975 non-null  object 
 2   Fulfilment          128975 non-null  object 
 3   ship-service-level  128975 non-null  object 
 4   Category            128975 non-null  object 
 5   Qty                 128975 non-null  int64  
 6   Amount              121180 non-null  float64
 7   ship-city           128942 non-null  object 
 8   ship-state          128942 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 8.9+ MB


here "Date" is in object Dtype --> change to Date Dtype

In [8]:
wdf['Date'] = pd.to_datetime(wdf['Date'])
wdf['Month_Year'] = wdf['Date'].dt.to_period('M')
wdf.drop('Date', axis=1, inplace=True)
wdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype    
---  ------              --------------   -----    
 0   Status              128975 non-null  object   
 1   Fulfilment          128975 non-null  object   
 2   ship-service-level  128975 non-null  object   
 3   Category            128975 non-null  object   
 4   Qty                 128975 non-null  int64    
 5   Amount              121180 non-null  float64  
 6   ship-city           128942 non-null  object   
 7   ship-state          128942 non-null  object   
 8   Month_Year          128975 non-null  period[M]
dtypes: float64(1), int64(1), object(6), period[M](1)
memory usage: 8.9+ MB


/tmp/ipykernel_17/2458292489.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wdf['Date'] = pd.to_datetime(wdf['Date'])


In [9]:
wdf.isna().sum()

Status                   0
Fulfilment               0
ship-service-level       0
Category                 0
Qty                      0
Amount                7795
ship-city               33
ship-state              33
Month_Year               0
dtype: int64

In [10]:
wdf = wdf.replace("None",np.nan)
wdf.isna().sum()

Status                   0
Fulfilment               0
ship-service-level       0
Category                 0
Qty                      0
Amount                7795
ship-city               33
ship-state              33
Month_Year               0
dtype: int64

In [11]:
wdf.describe()

,Qty,Amount
count,128975.000000,121180.000000
mean,0.904431,648.561465
std,0.313354,281.211687
min,0.000000,0.000000
25%,1.000000,449.000000
50%,1.000000,605.000000
75%,1.000000,788.000000
max,15.000000,5584.000000


In [12]:
wdf[wdf["Amount"].isna()]["Status"].value_counts()

Status
Cancelled                       7566
Shipped                          208
Shipped - Delivered to Buyer       8
Shipping                           8
Shipped - Returned to Seller       3
Pending                            2
Name: count, dtype: int64

In [13]:
wdf['Status'].value_counts()

Status
Shipped                          77804
Shipped - Delivered to Buyer     28769
Cancelled                        18332
Shipped - Returned to Seller      1953
Shipped - Picked Up                973
Pending                            658
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipping                             8
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64

## 4. Revenue Definition & Order Segmentation

To avoid revenue distortion, orders were segmented into:
- **Revenue-generating orders** (successfully fulfilled)
- **Revenue leakage orders** (cancelled, returned, lost, damaged)

This separation ensures accurate revenue measurement and enables focused leakage analysis.

In [14]:
revenue_status = [
    "Shipped",
    "Shipped - Delivered to Buyer",
    "Shipped - Picked Up",
    "Shipped - Out for Delivery",
    "Shipping"
]
leakage_status = [
    'Cancelled',
    'Shipped - Returned to Seller',
    'Pending',
    'Shipped - Returning to Seller',
    'Shipped - Rejected by Buyer',
    'Shipped - Lost in Transit',
    'Shipped - Damaged'
]
cols_needed = [
    'Status',
    'ship-service-level',
    'Category',
    'Qty',
    'Amount',
    'ship-city',
    'ship-state',
    'Month_Year'
]

In [15]:
revenue_df = wdf[
    wdf['Status'].isin(revenue_status)
][cols_needed].copy()
leakage_df = wdf[
    wdf['Status'].isin(leakage_status)
][cols_needed].copy()

In [16]:
revenue_df.head()

,Status,ship-service-level,Category,Qty,Amount,ship-city,ship-state,Month_Year
1,Shipped - Delivered to Buyer,Standard,kurta,1,406.0,BENGALURU,KARNATAKA,2022-04
2,Shipped,Expedited,kurta,1,329.0,NAVI MUMBAI,MAHARASHTRA,2022-04
4,Shipped,Expedited,Top,1,574.0,CHENNAI,TAMIL NADU,2022-04
5,Shipped,Expedited,Set,1,824.0,GHAZIABAD,UTTAR PRADESH,2022-04
6,Shipped,Expedited,Set,1,653.0,CHANDIGARH,CHANDIGARH,2022-04


In [17]:
leakage_df.head()

,Status,ship-service-level,Category,Qty,Amount,ship-city,ship-state,Month_Year
0,Cancelled,Standard,Set,0,647.62,MUMBAI,MAHARASHTRA,2022-04
3,Cancelled,Standard,Western Dress,0,753.33,PUDUCHERRY,PUDUCHERRY,2022-04
8,Cancelled,Expedited,Set,0,NaN,HYDERABAD,TELANGANA,2022-04
23,Cancelled,Standard,Set,0,570.48,pune,MAHARASHTRA,2022-04
29,Cancelled,Standard,kurta,0,NaN,GUWAHATI,ASSAM,2022-04


In [18]:
revenue_df['Status'].value_counts()

Status
Shipped                         77804
Shipped - Delivered to Buyer    28769
Shipped - Picked Up               973
Shipped - Out for Delivery         35
Shipping                            8
Name: count, dtype: int64

In [19]:
leakage_df['Status'].value_counts()

Status
Cancelled                        18332
Shipped - Returned to Seller      1953
Pending                            658
Shipped - Returning to Seller      145
Shipped - Rejected by Buyer         11
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64

In [20]:
len(revenue_df),len(leakage_df)

(107589, 21105)

In [21]:
revenue_df.describe()

,Qty,Amount
count,107589.00000,107365.000000
mean,1.00303,648.845462
std,0.09940,282.892833
min,0.00000,0.000000
25%,1.00000,449.000000
50%,1.00000,603.000000
75%,1.00000,788.000000
max,15.00000,5584.000000


In [22]:
leakage_df.describe()

,Qty,Amount
count,21105.000000,13534.000000
mean,0.400426,645.577605
std,0.495860,267.353606
min,0.000000,0.000000
25%,0.000000,442.000000
50%,0.000000,612.000000
75%,1.000000,771.000000
max,4.000000,4235.720000


## 5. Exploratory Data Analysis (EDA)

The EDA focuses on identifying revenue concentration across:
- Product categories
- Geographic regions (states)
- Order value distribution

In [23]:
category_revenue = (
    revenue_df.groupby('Category')['Amount']
    .sum()
    .sort_values(ascending=False)
)

(category_revenue.head(3) / category_revenue.sum()) * 100


Category
Set              49.862466
kurta            27.174489
Western Dress    14.108931
Name: Amount, dtype: float64

**Insight:**  
Revenue is highly concentrated, with the top three categories contributing approximately 90% of total revenue.  
This indicates significant dependency on a narrow product mix, increasing revenue risk.


In [24]:
state_aov = (
    revenue_df
    .groupby('ship-state')
    .agg(
        total_amount=('Amount', 'sum'),
        total_qty=('Qty', 'sum')
    )
)

state_aov['AOV'] = state_aov['total_amount'] / state_aov['total_qty']

state_aov.sort_values(by='AOV', ascending=False).head(5)


,total_amount,total_qty,AOV
ship-state,,,
bihar,1432.0,1,1432.000000
LADAKH,31229.0,34,918.500000
Sikkim,2703.0,3,901.000000
Chandigarh,7752.0,9,861.333333
LAKSHADWEEP,2441.0,3,813.666667


**Insight:**  
While high-volume states dominate total revenue, several low-volume states exhibit significantly higher average order values, indicating pockets of premium demand.


In [25]:
total_revenue = revenue_df['Amount'].sum()
total_qty = revenue_df['Qty'].sum()
avg_val = revenue_df['Amount'].mean()
median_val = revenue_df['Amount'].median()

total_revenue,total_qty,avg_val,median_val

(np.float64(69663293.0),
 np.int64(107915),
 np.float64(648.8454617426536),
 603.0)

In [26]:
revenue_df['Amount'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])


count    107365.000000
mean        648.845462
std         282.892833
min           0.000000
25%         449.000000
50%         603.000000
75%         788.000000
90%        1075.000000
95%        1186.000000
99%        1442.000000
max        5584.000000
Name: Amount, dtype: float64

## 6. Outlier Analysis (High-Value Orders)

Order value distribution is right-skewed, driven by a small number of high-value transactions.  
The IQR method is used to identify statistically extreme but potentially valuable orders.

In [27]:
q1 = revenue_df['Amount'].quantile(0.25)
q3 = revenue_df['Amount'].quantile(0.75)

iqr = q3 - q1

lower_bound = q1-1.5*iqr
upper_bound = q3+1.5*iqr

q1,q3,iqr,lower_bound, upper_bound

(np.float64(449.0),
 np.float64(788.0),
 np.float64(339.0),
 np.float64(-59.5),
 np.float64(1296.5))

In [28]:
outlier_df = revenue_df[
    (revenue_df['Amount'] < lower_bound) |
    (revenue_df['Amount'] > upper_bound)
]

len(outlier_df) , len(revenue_df)

(3266, 107589)

In [29]:
len(outlier_df) / len(revenue_df) * 100

3.035626318675701

In [30]:
outlier_df['Category'].value_counts()

Category
Set              3169
Western Dress      46
kurta              27
Ethnic Dress       18
Top                 3
Saree               3
Name: count, dtype: int64

In [31]:
outlier_df.groupby('ship-state')['Amount'].mean().sort_values(ascending = False).head(5)

ship-state
NAGALAND           1745.666667
Chandigarh         1556.000000
PUNJAB             1536.702128
ANDHRA PRADESH     1483.219697
JAMMU & KASHMIR    1471.681818
Name: Amount, dtype: float64

In [32]:
(revenue_df['Qty'].sum(), revenue_df['Amount'].sum())

(np.int64(107915), np.float64(69663293.0))

In [33]:
(outlier_df['Qty'].sum(), outlier_df['Amount'].sum())

(np.int64(3439), np.float64(4682454.0))

In [34]:
qty_per = 100*(outlier_df['Qty'].sum())/(revenue_df['Qty'].sum())
amt_per = 100*(outlier_df['Amount'].sum())/(revenue_df['Amount'].sum())
qty_per,amt_per

(np.float64(3.1867673632025206), np.float64(6.72155133407202))

**Insight:**  
Approximately 3% of orders account for nearly 7% of total revenue.  
These outliers represent valid high-value purchases and were retained for analysis as a premium customer segment.

## 7. Revenue Decomposition

Revenue is decomposed into its core drivers to identify which lever has the highest impact:

\[\text{Net Revenue} \approx (\text{Orders} \times \text{AOV}) - \text{Leakage}\]

This enables comparison between growth (AOV uplift) and efficiency (leakage reduction).


In [35]:
# Fulfilled (revenue-generating) orders
total_orders = revenue_df.shape[0]          # number of orders
total_qty = revenue_df['Qty'].sum()          # total units sold
total_revenue = revenue_df['Amount'].sum()   # gross revenue

aov = total_revenue / total_orders           # Average Order Value

total_orders, total_qty, total_revenue, aov


(107589,
 np.int64(107915),
 np.float64(69663293.0),
 np.float64(647.4945672884775))

In [36]:
outlier_orders = outlier_df.shape[0]
outlier_qty = outlier_df['Qty'].sum()
outlier_revenue = outlier_df['Amount'].sum()

outlier_order_pct = (outlier_orders / total_orders) * 100
outlier_revenue_pct = (outlier_revenue / total_revenue) * 100

outlier_orders, outlier_revenue, outlier_order_pct, outlier_revenue_pct

(3266, np.float64(4682454.0), 3.035626318675701, np.float64(6.72155133407202))

In [37]:
leakage_orders = leakage_df.shape[0]
leakage_qty = leakage_df['Qty'].sum()
leakage_revenue = leakage_df['Amount'].sum()

potential_revenue = total_revenue + leakage_revenue

leakage_order_pct = (leakage_orders / (leakage_orders + total_orders)) * 100
leakage_revenue_pct = (leakage_revenue / potential_revenue) * 100

leakage_orders, leakage_revenue, leakage_order_pct, leakage_revenue_pct

(21105,
 np.float64(8737247.299999999),
 16.399365937805957,
 np.float64(11.144371284390242))

## 8. Scenario Simulation (What-If Analysis)

Multiple business scenarios were simulated to compare revenue impact across different strategic levers.

In [38]:
leakage_reduction_rate = 0.20

revenue_recovered_from_leakage = leakage_revenue * leakage_reduction_rate
revenue_recovered_from_leakage

np.float64(1747449.46)

In [39]:
aov_increase_rate = 0.05

delta_aov = aov * aov_increase_rate
revenue_gain_from_aov = total_orders * delta_aov

revenue_gain_from_aov

np.float64(3483164.65)

In [40]:
revenue_loss_if_outliers_lost = outlier_revenue
revenue_loss_if_outliers_lost

np.float64(4682454.0)

In [41]:
scenario_summary = pd.DataFrame({
    "Scenario": [
        "Reduce leakage by 20%",
        "Increase AOV by 5%",
        "Lose all outlier orders"
    ],
    "Revenue Impact (INR)": [
        revenue_recovered_from_leakage,
        revenue_gain_from_aov,
        -revenue_loss_if_outliers_lost
    ]
})

scenario_summary

,Scenario,Revenue Impact (INR)
0,Reduce leakage by 20%,1747449.46
1,Increase AOV by 5%,3483164.65
2,Lose all outlier orders,-4682454.00


**Key Finding:**  
A 5% increase in Average Order Value generates significantly higher revenue impact than a 20% reduction in leakage, indicating that value uplift is a stronger growth lever than loss prevention.

## 9. Executive Decision & Recommendations

### Final Decision:
Prioritize increasing Average Order Value through premium product mix optimization, while selectively protecting high-value orders from leakage.

### Why This Strategy:
- AOV uplift provides higher revenue upside
- Product mix and pricing are highly controllable levers
- Leakage disproportionately impacts high-value orders

## Final Executive Summary (1-Minute Read)

### Problem
Revenue sustainability risk due to cancellations, returns, and dependence on a limited set of high-performing categories.

### Key Insights
- Revenue is highly concentrated: ~3% of orders contribute ~7% of total revenue.
- Revenue leakage exists but contributes less upside than value-based growth.
- Scenario analysis shows a 5% AOV increase generates higher revenue impact than a 20% leakage reduction.

### Decision
Prioritize AOV growth through premium product mix optimization, while selectively protecting high-value orders from revenue leakage.

### Why This Decision
This approach maximizes revenue impact using the most controllable business lever and avoids over-investing in low-return efficiency improvements.

### Next Actions
- Pilot premium bundles and pricing experiments
- Strengthen fulfilment controls for high-AOV orders
- Track AOV, leakage, and high-value order performance weekly

## Assumptions, Risks & Limitations

- Revenue is calculated using order-level gross amount; cost and margin data were not available.
- Leakage recovery was assumed to be partially controllable (20%) for scenario analysis.
- AOV uplift scenarios assume no significant negative demand elasticity.
- Analysis is based on historical data and does not account for future market changes.
- Results are specific to the analyzed time period and geography (India).